In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import os
os.chdir('/home/mathieu/chuvdocs/notebook/ml_labs/project1/')

## Load the training data into feature matrix, class labels, and event ids:

In [9]:
from proj1_helpers import *
DATA_TRAIN_PATH = '' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data('train.csv')
print(y.shape, tX.shape)

(250000,) (250000, 30)


## Do your thing crazy machine learning thing here :) ...

Define functions:

In [3]:
def compute_mse(y, tx, w):
    return np.mean(np.square(y-np.dot(tx, w)))

def compute_rmse(y, tx, w):
    return np.sqrt(compute_mse(y, tx, w))

def least_squares(y, tx):
    """Calculate the least squares solution. Returns weights."""
    
    H=np.dot(np.linalg.inv(np.dot(tx.T, tx)), tx.T)
    return np.dot(H, y)

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""

    return np.power(x.reshape(-1, 1), np.arange(degree+1))

def split_data(x, y, ratio, seed=1):
    """Split the dataset based on the split ratio. ratio is the relative size of the training set."""
    
    # Set seed
    np.random.seed(seed)
    
    # Get random incices
    idx = np.random.permutation(y.size)
    
    # Split indices into training and testing sets
    ksplit = int(idx.size*ratio)
    idx_tr, idx_te = idx[:ksplit], idx[ksplit:]
    assert idx.size == idx_tr.size+idx_te.size
    
    return (x[idx_tr], y[idx_tr]), (x[idx_te], y[idx_te])

def ridge_regression(y, tx, lamb):
    """Implement ridge regression."""
    
    lambd = lamb*2*y.size
    return np.dot(np.dot(np.linalg.inv(np.dot(tx.T, tx)+lambd*np.eye(tx.shape[1])), tx.T), y)
    

Test ridge regression:

In [8]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-3, 1, 10)
    
    # Split the data, and return train and test data
    (x_tr, y_tr), (x_te, y_te) = split_data(x, y, ratio, seed=seed)
    
    # Form train and test data with polynomial basis function
    x_tr = build_poly(x_tr, degree)
    x_te = build_poly(x_te, degree)

    # Ridge regression with different lambda
    rmse_tr = list()
    rmse_te = list()
    for lamb in lambdas:
        weight_tr = ridge_regression(y_tr, x_tr, lamb)
        weight_te = ridge_regression(y_te, x_te, lamb)
    
        # Calculate RMSE for train and test data
        rmse_tr.append(compute_rmse(y_tr, x_tr, weight_tr))
        rmse_te.append(compute_rmse(y_te, x_te, weight_te))
        
    plt.semilogx(lambdas, rmse_tr, marker='o', label='Train')
    plt.semilogx(lambdas, rmse_te, marker='o', label='Test')
    plt.legend()
    plt.xlabel('$\lambda$')
    plt.ylabel('RMSE')
    plt.show()

seed = 6
degree = 7
split_ratio = 0.5
ridge_regression_demo(tX, y, degree, split_ratio, seed)

ValueError: shapes (8,3750000) and (125000,) not aligned: 3750000 (dim 1) != 125000 (dim 0)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)